This script tested the toolbox for subplate-relevant genes. Here, genes are used in original order of DK atlas, not sorted.

In matlab, 20 genes were significant for CT, 18 for GYR. However, all of them had p_spin of 0. Is that significant or just some mistake?!
Test this in python...

In [28]:
import os
import enigmatoolbox
import numpy as np
import pandas as pd

from scipy.stats.mstats import spearmanr, pearsonr
from statsmodels.stats.multitest import fdrcorrection

In [29]:
# Import tmaps, ordered as in original AHBA
os.chdir(r'C:\Users\Acer\Documents\Studium\PhD\01_MA_preterm_gene-expression\analysis\data\tmaps\entrez_compatible')
CT = pd.read_csv('T_lhrh_thickness_term-preterm.csv')
GYR = pd.read_csv('T_lhrh_gyrification_term-preterm.csv')
CT.head()

,lh_bankssts_thickness,lh_caudalanteriorcingulate_thickness,lh_caudalmiddlefrontal_thickness,lh_cuneus_thickness,lh_entorhinal_thickness,lh_fusiform_thickness,lh_inferiorparietal_thickness,lh_inferiortemporal_thickness,lh_isthmuscingulate_thickness,lh_lateraloccipital_thickness,...,rh_rostralanteriorcingulate_thickness,rh_rostralmiddlefrontal_thickness,rh_superiorfrontal_thickness,rh_superiorparietal_thickness,rh_superiortemporal_thickness,rh_supramarginal_thickness,rh_frontalpole_thickness,rh_temporalpole_thickness,rh_transversetemporal_thickness,rh_insula_thickness
0,-0.051,0.042,0.059,0.042,-0.065,-0.027,-0.079,-0.027,-0.123,-0.023,...,-0.025,0.066,0.023,-0.067,-0.016,-0.063,-0.127,-0.234,0.011,-0.034


In [30]:
# Import gene expression data, ordered as in original AHBA
os.chdir(r'C:\Users\Acer\Documents\Studium\PhD\01_MA_preterm_gene-expression\analysis\data\microarray')
genes = pd.read_csv('expression_brainorder.csv', index_col=0).T
reglabels = list(genes.index)
genelabels = list(genes.columns)
print("Your input gene list contains expression data for", len(genelabels), "across", len(reglabels), "regions...")
genes.head()

Your input gene list contains expression data for 15633 across 68 regions...


gene_symbol,A1BG,A1BG-AS1,A2M,A2ML1,A3GALT2,A4GALT,AAAS,AACS,AADACL3,AADAT,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
L_bankssts,0.440726,0.589278,0.381510,0.436797,0.467330,0.483065,0.452128,0.543946,0.536256,0.588966,...,0.486024,0.533246,0.522263,0.601487,0.587398,0.405509,0.634875,0.497207,0.666547,0.477138
L_caudalanteriorcingulate,0.678336,0.431307,0.572806,0.671694,0.480824,0.503408,0.355954,0.368647,0.556326,0.584441,...,0.387346,0.363805,0.343892,0.549072,0.387021,0.633337,0.349052,0.223801,0.406646,0.503784
L_caudalmiddlefrontal,0.520608,0.446375,0.609580,0.459314,0.501768,0.542337,0.516451,0.527325,0.416930,0.433428,...,0.486235,0.506752,0.408213,0.448193,0.453520,0.438250,0.429106,0.587044,0.404144,0.529791
L_cuneus,0.317040,0.307230,0.451889,0.477776,0.474542,0.471519,0.687771,0.399432,0.566165,0.440505,...,0.590275,0.526024,0.370078,0.550802,0.423469,0.451572,0.444684,0.829026,0.733465,0.387726
L_entorhinal,0.711983,0.261514,0.699565,0.514992,0.455938,0.476981,0.313874,0.296195,0.600913,0.489880,...,0.368767,0.372662,0.779778,0.435531,0.376815,0.562862,0.542200,0.085269,0.453389,0.591145


In [31]:
# convert to numpy array
genes_np = genes.to_numpy()
CT_np = CT.to_numpy()
CT_np = np.squeeze(CT_np)
GYR_np = GYR.to_numpy()
GYR_np = np.squeeze(GYR_np)

# Thickness

### CT significant genes from matlab

In [32]:
mlGenesCT = ['ADAM15', 'CGN', 'CHADL', 'DDX39A', 'DNM2', 'GLYCTK', 'GPR150', 'GRIA2', 'HOMER3', 'IRAK4', 'LYG1', 'MEAF6',
            'OR7E156P', 'PTPRCAP', 'RNF38','SLC9A6','SMKR1', 'TSHZ2','ZNF681']
len(mlGenesCT)

19

In [33]:
from enigmatoolbox.permutation_testing import spin_test, perm_sphere_p
# generate empty array with 3 columns for gene index, Spearman'r, pval, pval_fdr, pval_spin
numGenes = len(mlGenesCT)
corr_coeffs = np.zeros([numGenes,6])

n = 0
for g in mlGenesCT:
    print("Performing correlation for gene:", g, "...")
    geneHere = genes.loc[:,g]
    coef, p = spearmanr(geneHere, CT_np)
    p_spin = spin_test(geneHere, CT_np, surface_name='fsa5', parcellation_name='aparc', type='spearman', n_rot=1000, null_dist=False)
    print('stop')
    
    #save values in array
    corr_coeffs[n,0] = n
    corr_coeffs[n,1] = coef
    corr_coeffs[n,2] = p
    corr_coeffs[n,4] = p_spin
    n=n+1


Performing correlation for gene: ADAM15 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
stop
Performing correlation for gene: CGN ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
stop
Performing correlation for gene: CHADL ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
stop
Performing correlation for gene: DDX39A ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of

In [34]:
# view results
results = pd.DataFrame(corr_coeffs)
results.columns = ['gene_symbol', 'Spearman_r', 'p', 'p_fdr', 'p_spin', 'pspin_fdr']
results['gene_symbol'] = mlGenesCT[0:numGenes]

# adjust for FDR: pval
pval = results.iloc[:,2].values
pval_fdr = list(fdrcorrection(pval)[1])
results['p_fdr'] = pval_fdr

# adjust for FDR: pspin
pvalspin = results.iloc[:,4].values
pvalspin_fdr = list(fdrcorrection(pvalspin)[1])
results['pspin_fdr'] = pvalspin_fdr


results.head(20)
# get significant genes based on p_spin
results.sort_values(['pspin_fdr'], ascending=True)


,gene_symbol,Spearman_r,p,p_fdr,p_spin,pspin_fdr
9,IRAK4,0.392273,0.000938,0.001485,0.0000,0.000000
5,GLYCTK,0.445013,0.000143,0.000545,0.0000,0.000000
13,PTPRCAP,0.484754,0.000028,0.000343,0.0000,0.000000
12,OR7E156P,0.477348,0.000039,0.000343,0.0000,0.000000
10,LYG1,0.388818,0.001050,0.001521,0.0000,0.000000
1,CGN,0.439592,0.000176,0.000559,0.0005,0.000950
2,CHADL,0.398687,0.000759,0.001441,0.0005,0.000950
4,DNM2,0.409815,0.000519,0.001097,0.0005,0.000950
14,RNF38,-0.392770,0.000923,0.001485,0.0005,0.000950
8,HOMER3,0.373395,0.001711,0.001908,0.0005,0.000950


### all genes 

repeat calculations in python to evaluate the differences (esp. of those genes that had p_spin of 0) with matlab outcomes (in C:\Users\Acer\Documents\Studium\PhD\01_MA_preterm_gene-expression\Melissa1909_ENIGMA\matlab\results)

In [35]:
from enigmatoolbox.permutation_testing import spin_test, perm_sphere_p
numGenes = len(genes.T)

# generate empty array
corr_coeffs = np.zeros([numGenes,6])

for g in range(0, numGenes):
    print("Performing correlation for gene:", genelabels[g], "...")
    geneHere = genes.iloc[:,g]
    coef, p = spearmanr(geneHere, CT_np)
    p_spin = spin_test(geneHere, CT_np, surface_name='fsa5', parcellation_name='aparc', type='spearman', n_rot=1000, null_dist=False)
    
    #save values in array
    corr_coeffs[g,0] = int(g)
    corr_coeffs[g,1] = coef
    corr_coeffs[g,2] = p
    corr_coeffs[g,4] = p_spin

Performing correlation for gene: A1BG ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: A1BG-AS1 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: A2M ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: A2ML1 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutati

permutation 1000 of 1000
Performing correlation for gene: ABCA2 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ABCA3 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ABCA5 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ABCA6 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutat

permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ABCG4 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ABCG8 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ABHD1 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ABHD10 ...
permutation 100 of 1000
permutation 200 of 1000
permuta

permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ABLIM3 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ABR ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ABRACL ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ABRAXAS2 ...
permu

permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ACCS ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ACD ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ACE ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performi

...

Performing correlation for gene: ATP1B1 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ATP1B2 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ATP1B3 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ATP23 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permut

permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ATP5PO ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ATP5S ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ATP6AP1 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ATP6AP1L ...
permutation 100 of 1000
pe

permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ATP8B1 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ATP8B2 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: ATP8B3 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
p

permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: AURKAIP1 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: AURKB ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: AURKC ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
p

KeyboardInterrupt: 

In [ ]:
# view results
results = pd.DataFrame(corr_coeffs)
results.columns = ['gene_symbol', 'Spearman_r', 'p', 'p_fdr', 'p_spin', 'pspin_fdr']
results['gene_symbol'] = genelabels

# adjust for FDR: pval
pval = results.iloc[:,2].values
pval_fdr = list(fdrcorrection(pval)[1])
results['p_fdr'] = pval_fdr

# adjust for FDR: pspin
pvalspin = results.iloc[:,4].values
pvalspin_fdr = list(fdrcorrection(pvalspin)[1])
results['pspin_fdr'] = pvalspin_fdr


results.head(20)
# get significant genes based on p_spin
results.sort_values(['pspin_fdr'], ascending=True)

In [ ]:
os.chdir(r'C:\Users\Acer\Documents\Studium\PhD\01_MA_preterm_gene-expression\Melissa1909_ENIGMA\results')
results.to_csv('explorative_CT.csv')